In [1]:
import ee
import geopandas as gpd
import numpy as np

from tqdm import tqdm

ee.Initialize()

In [2]:
file = 'landfire_zones'

landfire_zones = gpd.read_file(file, layer='conus_mz_0k')
sierras = landfire_zones[landfire_zones['ZONE_NUM'] == 6]
sierras_polygon = ee.Geometry.Polygon(list(sierras.iloc[0].geometry.exterior.coords),
                                      proj=ee.Projection('EPSG:5070'),
                                      geodesic=False)

In [3]:


mtbs_bs = (ee.ImageCollection("USFS/GTAC/MTBS/annual_burn_severity_mosaics/v1")
           .filterMetadata('system:index', 'contains', 'CONUS'))
mtbs_bs = mtbs_bs.map(lambda img: img.remap([0, 1, 2, 3, 4, 5, 6], [0, 1, 2, 3, 4, 5, 0]))

size = mtbs_bs.size().getInfo()
severity_classes = mtbs_bs.first().getInfo()['properties']['Severity_class_values']
severity_class_names = mtbs_bs.first().getInfo()['properties']['Severity_class_names']

size, severity_classes, severity_class_names


(39,
 [0, 1, 2, 3, 4, 5, 6],
 ['Background',
  'Unburned to Low',
  'Low',
  'Moderate',
  'High',
  'Increased Greenness',
  'Non-Mapping Area'])

In [4]:
mtbs_fires = ee.FeatureCollection("USFS/GTAC/MTBS/burned_area_boundaries/v1")
sierra_fires = mtbs_fires.filterBounds(sierras_polygon)
sierra_fires.aggregate_histogram('Asmnt_Type').getInfo()

{'Extended': 422, 'Initial': 218, 'Initial (SS)': 16}

In [5]:
sierra_fires = (sierra_fires
                .filter(ee.Filter.eq('Asmnt_Type', 'Extended'))
                .map(lambda f: f.set('year', ee.Date(f.get('Ig_Date')).get('year')))
                .filter(ee.Filter.gte('year', 2015))
                )

sierra_fires.size().getInfo(), sierra_fires.aggregate_histogram('year').getInfo()

(86,
 {'2015': 8,
  '2016': 14,
  '2017': 15,
  '2018': 11,
  '2019': 5,
  '2020': 15,
  '2021': 13,
  '2022': 5})

In [20]:
def ratio(f):
    area = f.geometry().area()
    mtbs_year = mtbs_bs.filter(ee.Filter.calendarRange(f.get('year'), f.get('year'), 'year')).first()
    reducer = ee.Reducer.sum()
    area_img = ee.Image.pixelArea()
    hs = mtbs_year.eq(4).selfMask()
    hs_area = ee.Number(area_img.updateMask(hs).reduceRegion(reducer, f.geometry(), 30).get('area'))
    return f.set('total_area', area,
                 'hs_ratio', hs_area.divide(area),
                 'hs_area', hs_area)


sierra_fires = sierra_fires.map(ratio)

In [21]:
fire_sizes = sierra_fires.aggregate_array('BurnBndAc').getInfo()
fire_sizes.sort()
fire_sizes

[1006,
 1057,
 1063,
 1064,
 1102,
 1137,
 1168,
 1184,
 1204,
 1270,
 1313,
 1316,
 1477,
 1672,
 1700,
 1805,
 1996,
 2121,
 2150,
 2289,
 2315,
 2317,
 2448,
 2487,
 2633,
 2706,
 2828,
 2829,
 2977,
 3003,
 3034,
 3155,
 3508,
 4065,
 4264,
 4346,
 4390,
 4545,
 4980,
 5221,
 5556,
 5743,
 5990,
 6018,
 6163,
 6488,
 6955,
 7016,
 7224,
 7713,
 8053,
 8495,
 9011,
 9090,
 9685,
 9717,
 12765,
 14252,
 14416,
 16100,
 18485,
 18576,
 18640,
 19408,
 26437,
 27331,
 29150,
 29191,
 36151,
 36626,
 48066,
 58747,
 70171,
 72894,
 77858,
 83297,
 89960,
 97307,
 97831,
 146369,
 153687,
 174424,
 226583,
 316556,
 381441,
 979795]

In [22]:
percentile_value = [int(v) for v in np.percentile(fire_sizes, [25, 75])]


small_fires = (sierra_fires.filter(ee.Filter.lt('BurnBndAc', percentile_value[0]))
               .map(lambda f: f.set('size_class', 'small')))

medium_fires = (sierra_fires.filter(ee.Filter.And(ee.Filter.gte('BurnBndAc', percentile_value[0]),
                                                  ee.Filter.lt('BurnBndAc', percentile_value[1])))
                .map(lambda f: f.set('size_class', 'medium')))

large_fires = (sierra_fires.filter(ee.Filter.gte('BurnBndAc', percentile_value[1]))
               .map(lambda f: f.set('size_class', 'large')))

final_fires_fc = small_fires.merge(medium_fires).merge(large_fires)

percentile_value, final_fires_fc.aggregate_histogram('size_class').getInfo()

([2349, 24679], {'large': 22, 'medium': 42, 'small': 22})

In [23]:
seed = 42
test_small = (final_fires_fc
              .filter(ee.Filter.eq('size_class', 'small'))
              .randomColumn(seed=seed)
              .limit(5, 'random')
              .limit(1, 'hs_ratio', False)
              )

test_medium = (final_fires_fc
               .filter(ee.Filter.eq('size_class', 'medium'))
               .randomColumn(seed=seed)
               .limit(10, 'random')
               .limit(2, 'hs_ratio', False)
               )
test_large = (final_fires_fc
              .filter(ee.Filter.eq('size_class', 'large'))
              .randomColumn(seed=seed)
              .limit(5, 'random')
              .limit(1, 'hs_ratio', False)
              )

test_fires = test_small.merge(test_medium).merge(test_large)

test_fires.aggregate_array('BurnBndAc').getInfo(), test_fires.aggregate_array('hs_ratio').getInfo()

([1270, 19408, 2828, 77858],
 [0.5023241785604305,
  0.4244191088828868,
  0.27457931293742416,
  0.3210810159763357])

In [24]:
test_small_ids = test_small.aggregate_array('system:index')
test_medium_ids = test_medium.aggregate_array('system:index')
test_large_ids = test_large.aggregate_array('system:index')

remove_ids = test_small_ids.cat(test_medium_ids).cat(test_large_ids)

train_val_fires = final_fires_fc.filter(
    ee.Filter.inList('system:index', remove_ids).Not())

train_val_fires.size().getInfo()

82

In [25]:
def set_dates_recent_mode(feat: ee.Feature):
    feature = ee.Feature(feat)
    date = ee.Date(feature.get('Ig_Date'))

    pre_0d = date.advance(-365, 'day').format('YYYY-MM-dd')
    pre_90d = date.advance(-275, 'day').format('YYYY-MM-dd')
    post_1d = date.advance(1, 'day').format('YYYY-MM-dd')
    post_30d = date.advance(30, 'day').format('YYYY-MM-dd')   
    post_60d = date.advance(60, 'day').format('YYYY-MM-dd')   

    return feature.set(
        'ig_dt_str', date.format('YYYY-MM-dd'),
        'pre_0d', pre_0d,
        'pre_90d', pre_90d,
        'post_1d', post_1d,
        'post_30d', post_30d,
        'post_60d', post_60d,
    )

In [26]:
test_fires = test_fires.map(set_dates_recent_mode)
train_val_fires = train_val_fires.map(set_dates_recent_mode)

print(test_fires.first().getInfo()['properties'])

{'Asmnt_Type': 'Extended', 'BurnBndAc': 1270, 'BurnBndLat': '39.276', 'BurnBndLon': '-120.632', 'Comment': '', 'Event_ID': 'CA3927312065620180903', 'High_T': 590, 'Ig_Date': 1535958000000, 'IncGreen_T': -150, 'Incid_Name': 'NORTH', 'Incid_Type': 'Wildfire', 'Low_T': 60, 'Map_ID': 10012361, 'Map_Prog': 'MTBS', 'Mod_T': 312, 'NoData_T': -970, 'Perim_ID': '', 'Post_ID': '804403320190722', 'Pre_ID': '804403320180719', 'area': 5135856.8409830285, 'dNBR_offst': 35, 'dNBR_stdDv': 29, 'hs_area': 2579865.068850768, 'hs_ratio': 0.5023241785604305, 'ig_date_str': '2018-09-03', 'irwinID': '6D3B481B-596E-4C42-AA69-2D42F0F68558', 'post_1d': '2018-09-04', 'post_30d': '2018-10-03', 'post_60d': '2018-11-02', 'pre_0d': '2017-09-03', 'pre_90d': '2017-12-02', 'random': 0.0034932110754584134, 'size_class': 'small', 'total_area': 5135857, 'year': 2018}


In [27]:
def create_grid(feature, grid_size, proj):
    grid = ee.FeatureCollection(feature.geometry().coveringGrid(ee.Projection(proj), grid_size))
    grid = grid.map(lambda f: f.copyProperties(feature))
    return grid

In [28]:
test_fires_grid = test_fires.map(
    lambda f: create_grid(f, 255*30, 'EPSG:5070')).flatten()
train_val_fires_grid = train_val_fires.map(
    lambda f: create_grid(f, 255*30, 'EPSG:5070')).flatten()

In [29]:
train_val_fires_df = ee.data.computeFeatures({'expression': train_val_fires_grid,
                                        'fileFormat': 'GEOPANDAS_GEODATAFRAME'})\
                                            .set_crs("EPSG:4326")\
                                            .to_crs('EPSG:5070')
test_fires_df = ee.data.computeFeatures({'expression': test_fires_grid,
                                         'fileFormat': 'GEOPANDAS_GEODATAFRAME'})\
                                            .set_crs("EPSG:4326")\
                                            .to_crs('EPSG:5070')
train_val_fires_df.columns, train_val_fires_df, test_fires_df

(Index(['geometry', 'Asmnt_Type', 'BurnBndAc', 'BurnBndLat', 'BurnBndLon',
        'Comment', 'Event_ID', 'High_T', 'Ig_Date', 'IncGreen_T', 'Incid_Name',
        'Incid_Type', 'Low_T', 'Map_ID', 'Map_Prog', 'Mod_T', 'NoData_T',
        'Perim_ID', 'Post_ID', 'Pre_ID', 'area', 'dNBR_offst', 'dNBR_stdDv',
        'hs_area', 'hs_ratio', 'ig_date_str', 'irwinID', 'post_1d', 'post_30d',
        'post_60d', 'pre_0d', 'pre_90d', 'size_class', 'total_area', 'year'],
       dtype='object'),
                                               geometry Asmnt_Type  BurnBndAc  \
 0    POLYGON ((-1981349.987 1759499.44, -1981349.99...   Extended       1204   
 1    POLYGON ((-2004299.923 1728899.778, -2004299.9...   Extended       1805   
 2    POLYGON ((-2050200.178 1828349.221, -2050200.1...   Extended       1064   
 3    POLYGON ((-2096100.367 1904849.246, -2096100.3...   Extended       1102   
 4    POLYGON ((-2103750.288 1912499.337, -2103750.3...   Extended       1102   
 ..                       

In [31]:
train_val_fires_df.to_file('mtbs_severity/train_val/fires')
test_fires_df.to_file('mtbs_severity/test/fires')

C:\Users\scone\AppData\Local\Temp\ipykernel_36996\3422366597.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  train_val_fires_df.to_file('mtbs_severity/train_val/fires')
c:\Users\scone\miniconda3\envs\utils\Lib\site-packages\pyogrio\raw.py:709: RuntimeWarning: Normalized/laundered field name: 'ig_date_str' to 'ig_date_st'
  ogr_write(
c:\Users\scone\miniconda3\envs\utils\Lib\site-packages\pyogrio\raw.py:709: RuntimeWarning: Value 117838902.697390988 of field area of feature 196 not successfully written. Possibly due to too larger number with respect to field width
  ogr_write(
c:\Users\scone\miniconda3\envs\utils\Lib\site-packages\pyogrio\raw.py:709: RuntimeWarning: Value 117838902.697390988 of field area of feature 197 not successfully written. Possibly due to too larger number with respect to field width
  ogr_write(
c:\Users\scone\miniconda3\envs\utils\Lib\site-packages\pyogrio\raw.py:709: RuntimeWarning: Value 117838902.697

In [19]:
print(test_fires_df.iloc[15])

geometry       POLYGON ((-2004300.0176659797 1667699.94952192...
Asmnt_Type                                              Extended
BurnBndAc                                                  29191
BurnBndLat                                                35.791
BurnBndLon                                              -118.571
Comment                                                         
Event_ID                                   CA3574511858420160816
High_T                                                       410
Ig_Date                                            1471330800000
IncGreen_T                                                  -150
Incid_Name                                                 CEDAR
Incid_Type                                              Wildfire
Low_T                                                         70
Map_ID                                                  10005157
Map_Prog                                                    MTBS
Mod_T                    

In [17]:
def qa(img):
        """mask clouds, shadows, snow"""
        qa_band = img.select("Fmask")
        qa_flag = int('11111',2) 
        mask = qa_band.bitwiseAnd(qa_flag).eq(0)
        
        return img.updateMask(mask)
    
def get_composite(feat):
    pre_0d = ee.Date(feat.loc["pre_0d"])
    pre_90d = ee.Date(feat.loc["pre_90d"])
    post_1d = ee.Date(feat.loc["post_1d"])
    post_30d = ee.Date(feat.loc["post_30d"])
    post_60d = ee.Date(feat.loc["post_60d"])
    
    region = ee.Geometry.Polygon(list(feat.geometry.exterior.coords), proj=ee.Projection('EPSG:5070'), geodesic=False)

    bands_in = ['B2','B3','B4','B5','B6','B7']
    pre_bands_out = [f"{b}_pre" for b in bands_in]
    post1_bands_out = [f"{b}_post1" for b in bands_in]
    post2_bands_out = [f"{b}_post2" for b in bands_in]

    hlsl30 = (ee.ImageCollection("NASA/HLS/HLSL30/v002")
        .map(qa)
        .select(bands_in))
    
    pre = hlsl30.filterDate(pre_0d, pre_90d).median()
    post1 = hlsl30.filterDate(post_1d, post_30d).median()
    post2 = hlsl30.filterDate(post_30d, post_60d).median()
    
    return ee.Image(ee.ImageCollection.fromImages([pre, post1, post2])
                    .toBands()
                    .rename(pre_bands_out+post1_bands_out+post2_bands_out))\
                    .reproject("EPSG:5070", scale=30).clipToBoundsAndScale(region, scale=30)

def get_mtbs_label(feat):
    year = ee.Number(int(feat.loc["year"]))
    region = ee.Geometry.Polygon(list(feat.geometry.exterior.coords), proj=ee.Projection('EPSG:5070'), geodesic=False)
    severity_img = mtbs_bs.filter(ee.Filter.calendarRange(year, year, 'year')).first()
    return ee.Image(severity_img).reproject("EPSG:5070", scale=30).clipToBoundsAndScale(region, scale=30)

def compute_save_tifs(df, out_dir):
    for i, row in tqdm(df.iterrows(), total=df.shape[0]):
        comp = get_composite(row)
        label = get_mtbs_label(row)
        
        request = {'expression': comp,'fileFormat': 'GeoTIFF',}
        data = ee.data.computePixels(request)
        with open(f'{out_dir}/images/chip_{i:05d}.tif', 'wb') as f:
            f.write(data)
            
        request = {'expression': label,'fileFormat': 'GeoTIFF',}
        data = ee.data.computePixels(request)
        with open(f'{out_dir}/labels/chip_{i:05d}.tif', 'wb') as f:
            f.write(data)

In [18]:
# compute_save_tifs(train_val_fires_df, 'mtbs_severity/train_val')
# compute_save_tifs(test_fires_df, 'mtbs_severity/test')

  1%|          | 4/687 [00:09<25:53,  2.28s/it]